In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q ftp://mirror.klaus-uwe.me/apache/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init("/content/spark-2.4.7-bin-hadoop2.7")

from pyspark.mllib.recommendation import *
import random
from operator import *
from collections import defaultdict

In [ ]:
from pyspark import SparkContext, SparkConf
spark = SparkContext.getOrCreate()
spark.stop()
spark = SparkContext('local','Recommender')

In [ ]:
artistData = spark.textFile('/content/artist_data_small.txt').map(lambda s:(int(s.split("\t")[0]),s.split("\t")[1]))
artistAlias = spark.textFile('/content/artist_alias_small.txt')
userArtistData = spark.textFile('/content/user_artist_data_small.txt')

In [ ]:
# Split a sequence into seperate entities and store as int

userArtistData = userArtistData.map(lambda s:(int(s.split(" ")[0]),int(s.split(" ")[1]),int(s.split(" ")[2])))

# Create a dictionary of the 'artistAlias' dataset

artistAliasDictionary = {}
dataValue = artistAlias.map(lambda s:(int(s.split("\t")[0]),int(s.split("\t")[1])))
for temp in dataValue.collect():
    artistAliasDictionary[temp[0]] = temp[1]

# If artistid exists, replace with artistsid from artistAlias, else retain original
userArtistData = userArtistData.map(lambda x: (x[0], artistAliasDictionary[x[1]] if x[1] in artistAliasDictionary else x[1], x[2]))

# Create an RDD consisting of 'userid' and 'playcount' objects of original tuple

userSum = userArtistData.map(lambda x:(x[0],x[2]))
playCount1 = userSum.map(lambda x: (x[0],x[1])).reduceByKey(lambda a,b : a+b)
playCount2 = userSum.map(lambda x: (x[0],1)).reduceByKey(lambda a,b:a+b)
playSumAndCount = playCount1.leftOuterJoin(playCount2)


# Count instances by key and store in broadcast variable
playSumAndCount = playSumAndCount.map(lambda x: (x[0],x[1][0],int(x[1][0]/x[1][1])))

# Compute and display users with the highest playcount along with their mean playcount across artists

TopThree = playSumAndCount.top(3,key=lambda x: x[1])
for i in TopThree:
    print('User '+str(i[0])+' has a total play count of '+str(i[1])+' and a mean play count of '+str(i[2])+'.')


User 1059637 has a total play count of 674412 and a mean play count of 1878.
User 2064012 has a total play count of 548427 and a mean play count of 9455.
User 2069337 has a total play count of 393515 and a mean play count of 1519.


In [ ]:
# Split the 'userArtistData' dataset into training, validation and test datasets. Store in cache for frequent access

trainData, validationData, testData = userArtistData.randomSplit((0.4,0.4,0.2),seed=13)
trainData.cache()
validationData.cache()
testData.cache()

PythonRDD[21] at RDD at PythonRDD.scala:53

In [ ]:
def modelEval(model, dataset):
    
    # All artists in the 'userArtistData' dataset

    AllArtists = spark.parallelize(set(userArtistData.map(lambda x:x[1]).collect()))
    
    
    # Set of all users in the current (Validation/Testing) dataset

    AllUsers = spark.parallelize(set(dataset.map(lambda x:x[0]).collect()))
    
    
    # Create a dictionary of (key, values) for current (Validation/Testing) dataset

    ValidationAndTestingDictionary ={}
    for temp in AllUsers.collect():
        tempFilter = dataset.filter(lambda x:x[0] == temp).collect()
        for item in tempFilter:
            if temp in ValidationAndTestingDictionary:
                ValidationAndTestingDictionary[temp].append(item[1])
            else:
                ValidationAndTestingDictionary[temp] = [item[1]]
                    
    
    # Create a dictionary of (key, values) for training dataset

    TrainingDictionary = {}
    for temp in AllUsers.collect():
        tempFilter = trainData.filter(lambda x:x[0] == temp).collect()
        for item in tempFilter:
            if temp in TrainingDictionary:
                TrainingDictionary[temp].append(item[1])
            else:
                TrainingDictionary[temp] = [item[1]]
        
    
    # For each user, calculate the prediction score i.e. similarity between predicted and actual artists

    PredictionScore = 0.00
    for temp in AllUsers.collect():
        ArtistPrediction =  AllArtists.map(lambda x:(temp,x))
        ModelPrediction = model.predictAll(ArtistPrediction)
        tempFilter = ModelPrediction.filter(lambda x :not x[1] in TrainingDictionary[x[0]])
        topPredictions = tempFilter.top(len(ValidationAndTestingDictionary[temp]),key=lambda x:x[2])
        l=[]
        for i in topPredictions:
            l.append(i[1])
        PredictionScore+=len(set(l).intersection(ValidationAndTestingDictionary[temp]))/len(ValidationAndTestingDictionary[temp])    

        
    # Print average score of the model for all users for the specified rank

    print("The model score for rank "+str(model.rank)+" is ~"+str(PredictionScore/len(ValidationAndTestingDictionary)))

In [ ]:
rankList = [2,10,20]
for rank in rankList:
    model = ALS.trainImplicit(trainData, rank , seed=345)
    modelEval(model,validationData)

The model score for rank 2 is ~0.08082178719723072
The model score for rank 10 is ~0.09052071953413846
The model score for rank 20 is ~0.08225274139572855


In [ ]:
bestModel = ALS.trainImplicit(trainData, rank=10, seed=345)
modelEval(bestModel, testData)

The model score for rank 10 is ~0.060728260020964896


In [ ]:
TopFive = bestModel.recommendProducts(2064012,5)
for item in range(0,5):
    print("Artist "+str(item)+": "+artistData.filter(lambda x:x[0] == TopFive[item][1]).collect()[0][1])

Artist 0: Rage Against the Machine
Artist 1: Franz Ferdinand
Artist 2: U2
Artist 3: Pink Floyd
Artist 4: Led Zeppelin
